# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac).

[Stactools dataset packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat).

Stactools and its dataset packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

## 1. Using this notebook

The easiest way to use this notebook is to run it through `scripts/notebook`. This will create a Docker container already set up with the prerequisites and open the stactools dataset folder. Navigate to `docs/` to find this notebook.

If you wish to use this notebook outside of the container (such as on mybinder.org) then please install the prerequisites using:

In [ ]:
!pip install stactools-nrcan-spot-ortho

## 2. Using the stactools dataset package from the CLI

The first thing we can do is check that the `stac` CLI tool is installed and explore the options. Notice the inclusion of the command for this dataset package in stactools now.

In [32]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  copy              Copy a STAC Catalog
  describe          Prints out a list of all catalogs, collections and items
                    in this STAC.

  info              Display info about a static STAC catalog.
  layout            Reformat the layout of a STAC based on templating.
  merge             Merge items from one STAC into another.
  move-assets       Move or copy assets in a STAC to the Item locations.
  nrcan-spot-ortho  Commands for ortho SPOT 4 and 5 data over Canada.
  validate          Validate a stac object.
  version           Display version info.


You can now explore the STAC dataset package commands to ingest and describe the data

In [2]:
!stac nrcan-spot-ortho --help

Usage: stac spot [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  cogify-assets  Convert geotiff assets into cloud optimized geotiffs.
  convert-index  Convert ortho SPOT 4 and 5 index shapefile to STAC catalog.


And more specific help with an individual command:

In [31]:
!stac nrcan-spot-ortho convert-index --help

Usage: stac nrcan-spot-ortho convert-index [OPTIONS] INDEX ROOT_HREF

  Converts the SPOT Index shapefile to a STAC Catalog.

Options:
  -t, --test                      Run as a test. Doesn't require Geobase FTP
                                  access.

  -c, --catalog-type [ABSOLUTE_PUBLISHED|RELATIVE_PUBLISHED|SELF_CONTAINED]
  --help                          Show this message and exit.


Download and unzip the [Geobase Index shapefile](http://ftp.maps.canada.ca/pub/nrcan_rncan/image/spot/geobase_orthoimages/index/GeoBase_Orthoimage_Index.zip) before converting it to the STAC:

In [22]:
!wget http://ftp.maps.canada.ca/pub/nrcan_rncan/image/spot/geobase_orthoimages/index/GeoBase_Orthoimage_Index.zip
!unzip GeoBase_Orthoimage_Index.zip

--2021-08-16 14:32:09--  http://ftp.maps.canada.ca/pub/nrcan_rncan/image/spot/geobase_orthoimages/index/GeoBase_Orthoimage_Index.zip
Resolving ftp.maps.canada.ca (ftp.maps.canada.ca)... 15.222.205.109
Connecting to ftp.maps.canada.ca (ftp.maps.canada.ca)|15.222.205.109|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp.maps.canada.ca/pub/nrcan_rncan/image/spot/geobase_orthoimages/index/GeoBase_Orthoimage_Index.zip [following]
--2021-08-16 14:32:09--  https://ftp.maps.canada.ca/pub/nrcan_rncan/image/spot/geobase_orthoimages/index/GeoBase_Orthoimage_Index.zip
Connecting to ftp.maps.canada.ca (ftp.maps.canada.ca)|15.222.205.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 665095 (650K) [application/zip]
Saving to: ‘GeoBase_Orthoimage_Index.zip’

GeoBase_Orthoimage_ 100%[===================>] 649.51K  1.71MB/s    in 0.4s    

2021-08-16 14:32:10 (1.71 MB/s) - ‘GeoBase_Orthoimage_Index.zip’ saved [665095/665095]

Now convert the index to a STAC (this will take several minutes - there are around 5000 entries)

In [38]:
!stac nrcan-spot-ortho convert-index GeoBase_Orthoimage_Index.shp "path/to/root/"

Connecting to Geobase FTP, attempt 1/10
1... S5_09537_5435_20070531
2... S5_09018_5147_20060830
3... S5_11913_6124_20060612
4... S5_09544_4857_20060904
5... S5_11055_6057_20070622
^C

Aborted!


In [ ]:
!stac nrcan-spot-ortho cogify-assets "path/to/root/catalog.json"

## 3. Using the stactools dataset package from within Python code

So far we've used Jupyter Notebooks' IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line, but it's also possible to use them within Python scripts.

In [47]:
from stactools.nrcan_spot_ortho.stac import build_items
from stactools.nrcan_spot_ortho.stac_templates import build_root_catalog
from stactools.nrcan_spot_ortho.cog import cogify_catalog
from pystac import CatalogType

root_href = "."
index = "GeoBase_Orthoimage_Index.shp"
catalog_path = "path/to/root/catalog.json"

In [ ]:
# Create a root catalog and collections for each sensor
spot_catalog = build_root_catalog()
spot_catalog.normalize_hrefs(root_href)

In [ ]:
# Populate the catalog with items
build_items(
    index_geom=index, 
    spot_catalog=spot_catalog, 
    test=False, 
    root_href=root_href, 
    catalog_type=CatalogType.ABSOLUTE_PUBLISHED)
spot_catalog.normalize_and_save(root_href, CatalogType.ABSOLUTE_PUBLISHED)

In [ ]:
# Download, unzip and convert geotiffs to cloud optimized geotiffs
# Save these within the catalog
cogify_catalog(catalog_path)